In [5]:
import pickle, os, glob
import pandas as pd
from obspy import UTCDateTime
from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

In [6]:
def ReadPKL_manual(evtpath):
    Meta_pkl = evtpath + '/Meta_manual.pkl'
    Station_data_pkl = evtpath + '/Station_data.pkl'
    Split_result_pkl = evtpath + '/Split_results_manual.pkl'
    SC_result_pkl = evtpath + '/SC_res_manual.pkl'
    RC_result_pkl = evtpath + '/RC_res_manual.pkl'
#     pick_pkl = evtpath + '/Pickphase.pkl'
    Meta = pickle.load(open(Meta_pkl, "rb"))
    Station_data = pickle.load(open(Station_data_pkl, "rb"))
    Split_result = pickle.load(open(Split_result_pkl, "rb"))
    SC_res = pickle.load(open(SC_result_pkl, "rb"))
    RC_res = pickle.load(open(RC_result_pkl, "rb"))
#     Pick = pickle.load(open(pick_pkl, "rb"))
    return Meta, Station_data, Split_result, SC_res, RC_res
def ReadPKL_auto(evtpath):
    Meta_pkl = evtpath + '/Meta_data.pkl'
    Station_data_pkl = evtpath + '/Station_data.pkl'
    Split_result_pkl = evtpath + '/Split_results_auto.pkl'
    SC_result_pkl = evtpath + '/SC_res.pkl'
    RC_result_pkl = evtpath + '/RC_res.pkl'
    Meta = pickle.load(open(Meta_pkl, "rb"))
    Station_data = pickle.load(open(Station_data_pkl, "rb"))
    Split_result = pickle.load(open(Split_result_pkl, "rb"))
    SC_res = pickle.load(open(SC_result_pkl, "rb"))
    RC_res = pickle.load(open(RC_result_pkl, "rb"))
    return Meta, Station_data, Split_result, SC_res, RC_res
def Creatdf():
    file = {'station':[],'Baz':[],'Inc':[],'Event':[],'Ev_lat':[],'Ev_lon':[], 'Depth':[],'Mag':[],'Phase':[],
            'SCPhi':[],'SCPhi_std':[],'SCdt':[],'SCdt_std':[],
            'RCPhi':[],'RCPhi_std':[],'RCdt':[],'RCdt_std':[],
            'SNRQ':[],'SNRT':[],'Null':[],'Quality':[],'CpH':[],'Pick':[]}
    return file
def Append(df,baz,inc,ev_lat,ev_lon,dep,mag,phase,
           SCphi,SCphi_std,SCdt,SCdt_std, RCphi,RCphi_std,RCdt,RCdt_std,
           snrq,snrt,null,quality,CpH,pick=True):
    df['Baz'].append(baz)
    df['Inc'].append(inc)
    df['Ev_lat'].append(ev_lat)
    df['Ev_lon'].append(ev_lon)
    df['Depth'].append(dep)
    df['Mag'].append(mag)
    df['Phase'].append(phase)
    df['SCPhi'].append(SCphi)
    df['SCPhi_std'].append(SCphi_std)
    df['SCdt'].append(SCdt)
    df['SCdt_std'].append(SCdt_std)
    df['RCPhi'].append(RCphi)
    df['RCPhi_std'].append(RCphi_std)
    df['RCdt'].append(RCdt)
    df['RCdt_std'].append(RCdt_std)
    df['SNRQ'].append(snrq)
    df['SNRT'].append(snrt)
    df['Null'].append(null)
    df['Quality'].append(quality)
    df['CpH'].append(CpH)
    df['Pick'].append(pick)
def Calc_rho(RCdt, SCdt):
    rho = RCdt/SCdt 
    return rho
def Calc_Phi(RCPhi, SCPhi):
    Phi = max (abs(RCPhi-SCPhi), abs(SCPhi-RCPhi))
    if Phi > 90: Phi = 180 - Phi
    return Phi

In [51]:
DATApath = '/Volumes/home/Research/STEP/01_Split'
SAVEpath = '/Users/tong/sws_tools/01_stereoplots/00_data'
Path = '/Volumes/home/Research/STEP/02_Station_result_csv'
phase='SKS'

In [52]:
for STApath in glob.glob(f'{DATApath}/*.pkl'):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]
    
    NET = values['network']; STA = values['station']

    newdf = pd.DataFrame()
    Splitfile = Creatdf()
    Nullfile = Creatdf()
    for CSVpath in sorted(glob.glob(f'{Path}/*_SKS/*{NET}*{STA}*.csv')):
        df = pd.read_csv(CSVpath)
        df = df[df['Phase']==phase][df['SNRQ']>=5][df['Pick']==True]
        newdf = newdf.append(df)

    for i in range(len(newdf)):
        Event = newdf['Event'].values[i]
        RCPhi = newdf['RCPhi'].values[i]
        RCdt  = newdf['RCdt'].values[i]
        SCPhi = newdf['SCPhi'].values[i]
        SCdt  = newdf['SCdt'].values[i]
        CpH   = newdf['CpH'].values[i]

        rho = Calc_rho(RCdt, SCdt)
        phi = Calc_Phi(RCPhi, SCPhi)
        if 25 < phi < 68 or CpH  > 0.76 :
            if CpH  >=0.9 : NULL='True'; quality='good'
            else: NULL='True'; quality='fair'
        elif 0.8 < rho < 1.1 and phi < 8: NULL='False'; quality='good'
        elif 0.7 <= rho < 1.2 and phi <= 25: NULL='False'; quality='fair'
        else: NULL='True'; quality='poor'

        meta = pickle.load(open(glob.glob(f'{DATApath}/DATA_SKS*/{NET}.{STA}/{Event}/Meta_manual.pkl')[0], "rb"))
        arrivals = model.get_travel_times(source_depth_in_km=meta.dep,
                    distance_in_degree=meta.gac,phase_list=['SKS'])
        inc = round(arrivals[0].incident_angle,2)  
    
        if NULL == 'False':
                Append(Splitfile,round(meta.baz,2),inc,meta.lat,meta.lon,meta.dep,meta.mag,meta.phase,
                SCPhi,newdf['SCPhi_std'].values[i],SCdt,newdf['SCdt_std'].values[i],
                RCPhi,newdf['RCPhi_std'].values[i],RCdt,newdf['RCdt_std'].values[i],
                newdf['SNRQ'].values[i], newdf['SNRT'].values[i],NULL,quality,CpH,pick=True)
                Splitfile['Event'].append(Event)
                Splitfile['station'].append(f'{NET}.{STA}')
        elif NULL == 'True':
                Append(Nullfile,round(meta.baz,2),inc,meta.lat,meta.lon,meta.dep,meta.mag,meta.phase,
                SCPhi,newdf['SCPhi_std'].values[i],SCdt,newdf['SCdt_std'].values[i],
                RCPhi,newdf['RCPhi_std'].values[i],RCdt,newdf['RCdt_std'].values[i],
                newdf['SNRQ'].values[i], newdf['SNRT'].values[i],NULL,quality,CpH,pick=True)  
                Nullfile['Event'].append(Event)    
                Nullfile['station'].append(f'{NET}.{STA}')    
        
    filename = f'{SAVEpath}/splitresults_{NET}.{STA}.csv'
    df = pd.DataFrame(Splitfile)
    df.to_csv(filename,index=False)
    filename = f'{SAVEpath}/splitresultsNULL_{NET}.{STA}.csv'
    df = pd.DataFrame(Nullfile)
    df.to_csv(filename,index=False)
    

/var/folders/sp/pnngww1n0q31w2plgbk53rc00000gn/T/ipykernel_80683/2575553174.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df['Phase']==phase][df['SNRQ']>=5][df['Pick']==True]


In [45]:
newdf['Event'][7]

7    20130416_225527
7    20110620_163600
7    20160917_012017
7    20160831_031136
7    20190506_211937
Name: Event, dtype: object

In [44]:
for STApath in sorted(glob.glob(f'{DATApath}/IU*.pkl')):
    STApkl = pickle.load(open(STApath, "rb"))
    values = [value for keys, value in STApkl.items() ][0]
    
    NET = values['network']; STA = values['station']
    print(STA)
    Splitfile = Creatdf()
    Nullfile = Creatdf()
    for Phase in ['SKS']:
        for PHASEpath in sorted(glob.glob(f'{DATApath}/DATA_{Phase}_Mw*/{NET}.{STA}')):
            for EVTpath in sorted(glob.glob(f'{PHASEpath}/*')):
                EVT = EVTpath.rsplit('/')[-1]
                try: 
                    try :
                        Meta, Station_data, Split_result, SC_res, RC_res= ReadPKL_manual(EVTpath)
    #                     print('manual')
                        rho = Calc_rho(RC_res.dtt, SC_res.dtt)
                        phi = Calc_Phi(RC_res.phi, SC_res.phi)
                        if 25 < phi < 68 or Meta.CpH > 0.76 :
                            if Meta.CpH >=0.9 : NULL='True'; quality='good'
                            else: NULL='True'; quality='fair'
                        elif 0.8 < rho < 1.1 and phi < 8: NULL='False'; quality='good'
                        elif 0.7 <= rho < 1.2 and phi <= 25: NULL='False'; quality='fair'
                        else: NULL='True'; quality='poor'
                        arrivals = model.get_travel_times(source_depth_in_km=Meta.dep,
                                  distance_in_degree=Meta.gac,phase_list=['SKS'])
                        inc = round(arrivals[0].incident_angle,2)
                        if NULL == 'False':
                             Append(Splitfile,round(Meta.baz,2),inc,Meta.lat,Meta.lon,Meta.dep,Meta.mag,Meta.phase,
                                round(SC_res.phi,2),round(SC_res.ephi,2),round(SC_res.dtt,2),round(SC_res.edtt,2),
                                round(RC_res.phi,2),round(RC_res.ephi,2),round(RC_res.dtt,2),round(RC_res.edtt,2),
                                round(Meta.snrq,2), round(Meta.snrt,2),NULL,quality,round(Meta.CpH,2),pick=True)
                             Splitfile['Event'].append(EVT)
                             Splitfile['station'].append(f'{NET}.{STA}')
                        elif NULL == 'True':
                             Append(Nullfile,round(Meta.baz,2),inc,Meta.lat,Meta.lon,Meta.dep,Meta.mag,Meta.phase,
                                round(SC_res.phi,2),round(SC_res.ephi,2),round(SC_res.dtt,2),round(SC_res.edtt,2),
                                round(RC_res.phi,2),round(RC_res.ephi,2),round(RC_res.dtt,2),round(RC_res.edtt,2),
                                round(Meta.snrq,2), round(Meta.snrt,2),NULL,quality,round(Meta.CpH,2),pick=True)    
                             Nullfile['Event'].append(EVT)    
                             Nullfile['station'].append(f'{NET}.{STA}')                   
                    except:
                        pass
                except: print('============'+STA+EVT+'error ============')
    # filename = f'{SAVEpath}/splitresults_{NET}.{STA}.csv'
    # df = pd.DataFrame(Splitfile)
    # df.to_csv(filename,index=False)
    # filename = f'{SAVEpath}/splitresultsNULL_{NET}.{STA}.csv'
    # df = pd.DataFrame(Nullfile)
    # df.to_csv(filename,index=False)

GNI


In [33]:
meta

['/Volumes/home/Research/STEP/01_Split/DATA_SKS_Mw6_test/IU.GNI/20160804_141513/Meta_manual.pkl',
 '/Volumes/home/Research/STEP/01_Split/DATA_SKS_Mw6/IU.GNI/20160804_141513/Meta_manual.pkl',
 '/Volumes/home/Research/STEP/01_Split/DATA_SKS_Mw7/IU.GNI/20160804_141513/Meta_manual.pkl']

In [ ]:
df = pd.DataFrame(Splitfile)

In [ ]:
df

,station,Baz,Inc,Event,Ev_lat,Ev_lon,Depth,Mag,Phase,SCPhi,...,RCPhi,RCPhi_std,RCdt,RCdt_std,SNRQ,SNRT,Null,Quality,CpH,Pick
0,IU.GNI,270.97,5.99,20110306_123158,-18.2064,-69.5006,114.50,6.3,SKS,62.97,...,48.97,44.75,0.3,0.98,18.77,5.23,False,fair,0.75,True
1,IU.GNI,282.95,6.25,20120802_093831,-8.4540,-74.2726,148.80,6.0,SKS,54.95,...,51.95,44.75,1.1,0.98,8.71,6.63,False,good,0.71,True
2,IU.GNI,214.82,6.66,20140201_035845,-56.9339,-27.3456,142.90,6.2,SKS,2.82,...,-2.18,44.75,2.2,0.98,4.18,9.71,False,good,0.27,True
3,IU.GNI,264.59,5.91,20150211_185720,-23.0653,-66.6621,203.60,6.7,SKS,32.59,...,43.59,44.75,0.5,0.98,13.47,1.63,False,fair,0.66,True
4,IU.GNI,266.31,5.82,20150610_135208,-22.4098,-68.4519,121.80,6.1,SKS,51.31,...,45.31,44.75,0.7,0.98,14.32,1.29,False,good,0.44,True
5,IU.GNI,264.93,6.00,20160804_141513,-22.3087,-66.0261,270.00,6.2,SKS,54.93,...,45.93,44.75,0.7,0.98,11.17,5.47,False,fair,0.71,True
6,IU.GNI,265.06,5.82,20170415_081943,-23.2706,-67.7655,155.00,6.2,SKS,50.06,...,41.06,16.25,0.7,0.35,11.94,1.72,False,fair,0.73,True
7,IU.GNI,270.33,5.95,20180121_010642,-18.8524,-69.4564,116.00,6.3,SKS,64.33,...,50.33,44.75,0.6,0.98,16.54,1.94,False,fair,0.56,True
8,IU.GNI,291.49,6.35,20180907_021204,-2.3453,-78.8955,93.53,6.2,SKS,51.49,...,75.49,44.75,1.4,0.98,9.22,8.47,False,fair,0.65,True
9,IU.GNI,78.35,6.71,20190503_072529,-6.9280,160.1389,10.00,6.2,SKS,39.35,...,30.35,44.75,0.7,0.98,8.02,-2.76,False,fair,0.76,True
